## Алгоритм Киркпатрика

### Введение

   Алгоритм Киркпатрика решает задачу локализации точки в планарном графе. Однако от остальных алгоритмов он отличается тем, что работает с PSLG, представляющими собой триангуляцию (за $O(n\log{n})$ заметающей прямой можно триангулировать любой PSLG, так что это не ограничивает его использование). Эта особенность делает использование этого алгоритма или его модификаций крайне удобным в некоторых случаях , например, для динамической выпуклой оболочки.  
   
   Далее считаем, что дан PSLG, каждый фейс которого является треугольником.
   
<img width="300px" height="300px" src="slideshow/images/kirkpatrick1.png">
<center><font size=2>Рис. 1. Пример триагуляции </font></center>

### Идея алгоритма

   Давайте построим последовательность триангуляций $\left\{T_i\right\}$, где $T_i$-я триангуляция получается из $T_{i - 1}$ удалением некоторой доли точек. Каждый треугольник в $T_i$-й триангуляции знает, какие треугольники в $T_{i - 1}$-й он пересекает.
Локализовавшись в $T_i$-й триангуляции, мы сможем перейти к $T_{i - 1}$. Так, спускаясь все ниже и ниже по уровням, мы дойдем до искомой триангуляции и локализуемся в ней.
Вопрос только в том, как строить эти уровни так, чтобы их было не слишком много и чтобы переходить от уровня к уровню было не слишком сложно.

### Предобработка

   Перед тем как перейти непосредственно к алгоритму, поместим весь PSLG в бесконечный квадрат и триангулируем его.
Для случая, указаного выше, после этих операций PSLG будет выглядеть так:

<img width="400px" height="400px" src="slideshow/images/kirkpatrick2.png">
<center><font size=2>Рис. 2. Триангулияция бесконечного квадрата. </font></center>

В результате в PSLG добавятся треугольники, которых не было.
Пометим их как внешние, и, если точка локализуется в таком треугольнике, то мы будем знать, что точка не принадлежит никакому фейсу исходной PSLG.

   Теперь получим некоторые необходимые нам потом оценки.
Для каждого фейса посчитаем число ребер в PSLG.
Так как каждое ребро принадлежит ровно двум треугольникам получаем соотношение $2E = 3F$.
При подстановке в формулу Эйлера (для выпуклого многогранника $V - E + F = 2$) получаем, что $F = 2V - 4$ и $E = 3V - 6$.

### Структура данных
   
Итак, имеется триангуляция $G$.
Будем строить последовательность триангуляций $S_1, S_2, \dots, S_{h(N)}$, где $N$ – количество вершин, $h(N)$ – количество построенных триангуляций (позже уточним как выглядит эта функция), $S_1 = G$, а $S_i$ получается из $S_{i - 1}$ по следующим правилам:
* <b>Шаг 1.</b> 
Удалим некоторое количество независимых (попарно несмежных друг с другом) вершин (вершины бесконечного квадрата удалять нельзя) и инцидентные им ребра (от выбора этого множества напрямую зависит оптимальность алгоритма).
* <b>Шаг 2.</b> 
Построить триангуляцию получившихся в результате шага $1$ многоугольников.

Переход от $S_1$ к $S_2$ в нашем примере может выглядеть, как показано на рисунке $3$.
<img width="800px" height="800px" src="slideshow/images/step1.png">
<center><font size=2>Рис. 3. Переход от $S_1$ к $S_2$ </font></center>

Будем повторять эти шаги до тех пор, пока $S_{h(N)}$ не будет состоять из $O(1)$ треугольника.
Далее будем обозначать все треугольники как $R$, а также будем говорить, что треугольник $R_{j}$ принадлежит триангуляции $S_i$, если 
он был создан на <b>втором</b> шаге при построении этой триангуляции.

Теперь построим структуру данных $T$ для поиска.
Эта структура представляет собой направленный ацикличный граф, вершинами которого будут наши треугольники.
Определим эту структуру следующим образом.
Из треугольника $R_k$ будет вести ребро в треугольник $R_j$, если при построении $S_i$ из $S_{i-1}$ мы имеем:
* $R_j$ удалятся из $S_{i - 1}$ на первом шаге,
* $R_k$ создается в $S_{i}$ на втором шаге,
* $R_j \cap R_k \ne  \emptyset$.

Очевидно, что треугольники из $S_1$ (и только они) не имеют исходящих ребер.
Для примера с переходом от $S_1$ к $S_2$ структура имела бы вид, изображенный на рисунке $4$.
<img width="800px" height="800px" src="slideshow/images/step2.png">
<center><font size=2>Рис. 4. Пример локализационной структуры </font></center>

Имея такую структуру данных, легко написать функцию, которая по точке вернет треугольник, которому она принадлежит.

In [ ]:
def locate(root, point):
    """
    root – корень локализационной структуры
    point – точка запроса
    """
    
    def go(v):
        # Переберем все ребра из текущего узла и найдем какому треугольнику точка принадлежит
        for node in curr.edges:
            if point in node:
                return node
    
    curr = root
    # Если curr равно None, то мы находимся на самом нижнем уровне локализационной структуры
    while curr is not None:
        # Пока делаем переходы в локализационной структуре будем повторять следующий цикл
        result = curr.triangle
        curr = go(curr)
    
    return result

### Корректность алгоритма

> Алгоритм найдет треугольник, которому принадлежит точка.

$\triangleright$<br>
<div style="padding-left:40px">
Самая простая триангуляция $S_{h(N)}$ будет разбивать всё пространство на $O(1)$ треугольников.
Пусть точка запроса принадлежит треугольнику $S'$ из $S_{h(N)}$, и треугольнику $S$ из $S_1$.

Тогда в локализационной структуре по построению должен найтись путь из $S'$ в $S$.
Точка запроса пройдет в локализационной структуре ровно по этому пути, потому что на любом из уровней точка может принадлежать ровно одному треугольнику.
</div>
$\triangleleft$

### Упражнение

Дано два треугольника. Вам необходимо определить, пересекаются ли они или нет.

In [ ]:
from testing import test_triangles_intersection, intersect

def intersect_triangles(a, b):
    """
    Здесь должно быть ваше решение.
    
    Аргументы a, b - это треугольники, которые представляют из себя список из трех точек.
    
    Функция возвращает True, если треугольники пересекаются и False в противном случае.
    """
    return intersect(a, b)

test_triangles_intersection(intersect_triangles)

### Выбор множества удаляемых вершин

Как уже упоминалось, от выбора множества вершин триангуляции, которые будут удалены при построении $S_i$ по $S_{i-1}$, существенно зависит эффективность метода. Предположим, что можно выбрать это множество так, чтобы выполнялись следующие ''свойства'' ($N_i$ обозначает число вершин в $S_i$):

<b>Свойство 1:</b> $N_i \leqslant a_i N_{i-1}$, где $a_i \le a < 1$ для $i = 2,\dots , h(N)$.

<b>Свойство 2:</b> Каждый треугольник $R_j \in S_i$ пересекается не более чем с $H$ треугольниками из $S_{i-1}$ и наоборот.

Первое свойство немедленно влечет за собой следствие, что $h(N) \leqslant \left \lceil \log_{1/a}N \right \rceil = O(\log{N})$, поскольку при переходе от $S_{i-1}$ к $S_i$ удаляется по меньшей мере фиксированная доля вершин.

### Критерий выбора множества удаляемых вершин
Покажем теперь, что критерий выбора множества удаляемых вершин, удовлетворяющий вышеописанным свойствам, существует.
<br>
> Если на <b>шаге 1</b> построения последовательности триангуляции удалять несмежные вершины со степенью меньше $12$, то свойства, описанные выше, будут выполнены.

<br>$\triangleright$<br>
<div style="padding-left:40px">
<b>1.</b>
Ранее мы показали, что в триангуляции существует $3V - 6$ ребер.
Так как каждое ребро инцидентно двум вершинам, то сумма степеней всех вершин меньше $6V$.
Отсюда сразу следует, что не менее половины вершин имеет степень меньше $12$.

Пусть $M$ – число выбранных вершин.
Так как вершины бесконечного квадрата удалять нельзя, то всего кандидатов на удаление не менее
$$\frac{V}{2} - 4.$$

Поскольку каждому кандидату инцидентно не более $11$ ребер, то при удалении вершины в худшем случае блокируется $12$ кандидатов(сама вершина и инцидентные ей вершины), которых уже нельзя удалять.
То есть из $12$ кандидатов в худшем случае будет удален лишь $1$, значит
$$M \geqslant \left \lfloor \frac{1}{12}\left(\frac{N}{2} - 4\right) \right \rfloor \text.$$

Следовательно, $a \cong 1 - \frac{1}{24} < 0,959 < 1$, что доказывает справедливость <b>свойства 1</b>.
<br>
<b>2.</b> Выполнение второго свойства обеспечивается тривиально.
Поскольку удаление вершины со степенью меньше $12$ приводит к образованию многоугольника с числом вершин менее $12$, то каждый из удаленных треугольников пересекает не более $12 - 2 = 10 = H$ новых треугольников.
</div>
$\triangleleft$

   Теперь мы можем написать функцию нахождения необходимого независимого множества вершин.

In [ ]:
def get_independent_set(points):
    """
    Эта функция находит множество независимых вершин, которые потом будут удаляться из триангуляции.
    """
    result = set()
    for v in points:
        # Проверим критерий и то, что точка не принадлежит бесконечному прямоугольнику
        if v.degree() < 12 and not v.in_rectangle: 
            flag = True
            # Проверим, что вершина не смежна ни с какой другой из result
            for e in v.edges():
                if e.end in result:
                    flag = False
                    break
            if flag:
                result.insert(v)
    return result

### Сложность алгортма

> Алгоритм Киркпатрика требует $O(\log{n})$ времени на запрос, $O(n)$ памяти и $O(n \log{n})$ времени на для построения поисковой структуры.

$\triangleright$<br>
<div style="padding-left:40px">
Во время запроса на каждом уровне алгоритм проверяет какому из $O(1)$ треугольников принадлежит точка, тогда так как уровней всего $O(\log{n})$ очевидно, что запрос работает за $O(\log{n})$.

Теперь докажем оценку памяти. Заметим, что эта память используется для хранения узлов и указателей на их потомков. Из формулы Эйлера следует, что $S_i$ содержит $F_i < 2N_i$ треугольников. Число узлов в $T$, представляющих треугольники из $S_i$, не превосходит $F_i$ (только те треугольники, которые действительно принадлежат $S_i$, появляются на соответствующем «ярусе» $T$). Отсюда следует, что общее число узлов в $T$ меньше, чем
<br>
$$2\left(N_1 + N_2 + \dots + N_{h(N)}\right) \leqslant 2N_1\left(1 + a + a^2 + \dots + a^{h(N) - 1}\right) < \frac{2N}{1 - a}$$

По <b>свойству 2</b> каждый узел имеет не более $H$ указателей, поэтому в $T$ число указателей не более чем

$$\frac{2NH}{1-a}\text.$$

Отсюда сразу следует, что для хранения всех указателей и вообще всей локализационной структуры используется $O(N)$ памяти.

Чтобы доказать оценку времени на построение структуры, нужно доказать, что построение очередного слоя работает за $O(n)$: так как всего слоев $O(\log{n})$, это повлечет за собой искомую оценку.
<br>
Выбрать все вершины для удаления мы можем, очевидно, за $O(n)$. После удаления вершин останется $V$ многоугольников, которые нужно триангулировать. Однако заметим, что все эти многоугольники – <b>звездные</b>, поэтому каждый из них можно триангулировать за $O(m)$, где $m$ – число вершин в многоугольнике. Суммарное число вершин в многоугольниках – $O(n)$, поэтому они все будут триангулированы за $O(n)$.
</div>
$\triangleleft$